In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import requests
from bs4 import BeautifulSoup
import json
import os


In [3]:
def sort_key(url):
    return int(url.split('-')[-1])

In [14]:
archive_links = [
    # 'https://web.archive.org/web/20180531184723/https://www.chess.com/tournament/live/titled-tuesdays',
    # 'https://web.archive.org/web/20170502043615/https://www.chess.com/tournament/live/titled-tuesdays',
    # 'https://web.archive.org/web/20170404005219/https://www.chess.com/tournament/live/titled-tuesdays'
]


In [15]:
tourn_links = []

In [16]:

for i in archive_links:
  r = requests.get(i)
  soup = BeautifulSoup(r.content, 'html.parser')
  tourn_list = soup.find('table', class_ = 'table with-row-highlight tournaments-list')
  tourn_table_rows = tourn_list.find_all('tr')
  tourn_table_rows = tourn_table_rows[1:]
  for j in tourn_table_rows:
    tourn_link = j.find('a', class_='clickable-e-link')['href']
    parts = tourn_link.split("https://www.chess.com/")
    chess_com_link = "https://www.chess.com/" + parts[1]
    tourn_links.append(chess_com_link)

In [19]:
tourn_links = list(set(tourn_links))
tourn_links = sorted(tourn_links, key=sort_key, reverse=True)
tourn_links

['https://www.chess.com/tournament/live/-titled-tuesday-blitz-954936',
 'https://www.chess.com/tournament/live/-biggest-ever-titled-tuesday-952610',
 'https://www.chess.com/tournament/live/-titled-tuesday-blitz-948450',
 'https://www.chess.com/tournament/live/-titled-tuesday-blitz-919983',
 'https://www.chess.com/tournament/live/-masterclass-titled-tuesday-blitz-914248',
 'https://www.chess.com/tournament/live/-masterclass-titled-tuesday-blitz-913741',
 'https://www.chess.com/tournament/live/-masterclass-titled-tuesday-blitz-906896',
 'https://www.chess.com/tournament/live/-masterclass-titled-tuesday-blitz-899622',
 'https://www.chess.com/tournament/live/-masterclass-titled-tuesday-blitz-894012',
 'https://www.chess.com/tournament/live/-masterclass-titled-tuesday-blitz-878114',
 'https://www.chess.com/tournament/live/-masterclass-titled-tuesday-blitz-862996',
 'https://www.chess.com/tournament/live/-titled-tuesday-32-blitz-851498',
 'https://www.chess.com/tournament/live/-titled-tuesda

In [20]:
for url in tourn_links:
  parts = url.split('/')
  tournament_id = parts[-1]
  output_filename = f"/content/drive/MyDrive/TTChesscom/Ranks/{tournament_id}.json"
  if os.path.exists(output_filename):
    print("File already exists: " + output_filename)
    continue
  r = requests.get(url+'?&players=100')
  soup = BeautifulSoup(r.content, 'html.parser')
  i_p = soup.find('div', class_ = 'index-pagination')
  data_total_pages = 0
  if i_p:
    data_total_pages = int(i_p.find('div')['data-total-pages'])
  print("total pages: " + str(data_total_pages))

  ranks  = []
  rank = 0
  for i in range(data_total_pages):
    print('page: ' + str(i+1))
    r = requests.get(url+'?&players='+str(i+1))
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find('table', class_ = 'table-component tournaments-live-view-results-table tournaments-live-view-extra-borders')
    table_rows = table.find_all('tr')
    table_rows = table_rows[1:]
    for x in table_rows:
      rank +=1
      if rank%100 == 0:
        print("completed: "+str(rank))
      username = x.select_one('.user-tagline-username').get_text(strip=True)
      country = x.select_one('.country-flags-component')['v-tooltip']
      rating = x.select_one('.user-rating').get_text(strip=True).replace('(','').replace(')','')
      if (rating != 'Unrated'):
        rating = int(rating)
      title_element = x.select_one('.post-view-meta-title')
      title = title_element.get_text(strip=True) if title_element is not None else None
      score = float(x.select_one('.tournaments-live-view-total-score').get_text(strip=True))
      tie_break = float(x.select_one('.tournaments-live-view-tie-break').get_text(strip=True))
      wdb = x.find('div', class_='tournaments-live-view-total-score')['v-tooltip'].split(',')
      wins = int(wdb[0].strip().split()[0])
      draws = int(wdb[1].strip().split()[0])
      byes = int(wdb[2].strip().split()[0])
      player = {
      "rank": rank,
      "username": username,
      "country": country,
      "rating": rating,
      "title": title,
      "score": score,
      "tie_break": tie_break,
      "wins": wins,
      "draws": draws,
      "byes": byes
      }
      ranks.append(player)

  with open(output_filename, 'w') as json_file:
    json.dump(ranks, json_file, indent=4)
  print("written: " + output_filename)


File already exists: /content/drive/MyDrive/TTChesscom/Ranks/-titled-tuesday-blitz-954936.json
File already exists: /content/drive/MyDrive/TTChesscom/Ranks/-biggest-ever-titled-tuesday-952610.json
File already exists: /content/drive/MyDrive/TTChesscom/Ranks/-titled-tuesday-blitz-948450.json
File already exists: /content/drive/MyDrive/TTChesscom/Ranks/-titled-tuesday-blitz-919983.json
File already exists: /content/drive/MyDrive/TTChesscom/Ranks/-masterclass-titled-tuesday-blitz-914248.json
File already exists: /content/drive/MyDrive/TTChesscom/Ranks/-masterclass-titled-tuesday-blitz-913741.json
File already exists: /content/drive/MyDrive/TTChesscom/Ranks/-masterclass-titled-tuesday-blitz-906896.json
File already exists: /content/drive/MyDrive/TTChesscom/Ranks/-masterclass-titled-tuesday-blitz-899622.json
File already exists: /content/drive/MyDrive/TTChesscom/Ranks/-masterclass-titled-tuesday-blitz-894012.json
File already exists: /content/drive/MyDrive/TTChesscom/Ranks/-masterclass-titl